In [55]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from simple_salesforce import Salesforce
import pandas as pd
import numpy as np
from collections import defaultdict
import itertools
from concurrent import futures
import concurrent.futures

In [41]:
def get_object_details():
    
    object_ = input('Which Object are you checking?')

    # read in document which specifies which objects/ fields are supposed to be migrated
    # remove unneccesary columns and format appropriatly for script
    
    stm = pd.read_csv(r'C:\Users\jschlajo\Desktop\Migration_Docs\BSNA Source-to-Target Mapping.csv', encoding='latin-1')
    stm = stm[['Object Name (API)','Field Name (API)','Data Type','Field Type','In Page Layout ?','Fields Migrate to BSNA','Object Name (API).1','Field Name (API).1','Data Type.1','Field Type.1',]]
    stm.columns = ['ObjectName(API)E3','FieldName(API)E3','DataTypeE3','FieldTypeE3','In Page Layout ?','Fields Migrate to BSNA','ObjectName(API)BSNA','FieldName(API)BSNA','DataTypeBSNA','FieldTypeBSNA']
    reqFields = stm[(stm['ObjectName(API)BSNA'] == object_) & (stm['Fields Migrate to BSNA'] == 'KEEP') & (stm['FieldName(API)BSNA'].isna()==False)]
    
    return reqFields, object_

In [42]:
object_details = get_object_details()
required_fields = object_details[0]
object_ = object_details[1]

Which Object are you checking? Campaign


In [45]:
required_fields.head()

,ObjectName(API)E3,FieldName(API)E3,DataTypeE3,FieldTypeE3,In Page Layout ?,Fields Migrate to BSNA,ObjectName(API)BSNA,FieldName(API)BSNA,DataTypeBSNA,FieldTypeBSNA
561,Campaign,ActualCost,"Currency(18, 0)",Standard,True,KEEP,Campaign,ActualCost,"Currency(18, 0)",Standard
562,Campaign,AmountAllOpportunities,"Currency(18, 0)",Standard,True,KEEP,Campaign,AmountAllOpportunities,"Currency(18, 0)",Standard
563,Campaign,AmountWonOpportunities,"Currency(18, 0)",Standard,True,KEEP,Campaign,AmountWonOpportunities,"Currency(18, 0)",Standard
564,Campaign,BudgetedCost,"Currency(18, 0)",Standard,True,KEEP,Campaign,BudgetedCost,"Currency(18, 0)",Standard
566,Campaign,CampaignMemberRecordTypeId,Lookup(Record Type),Standard,False,KEEP,Campaign,CampaignMemberRecordTypeId,Lookup(Record Type),Standard


In [115]:
def bsna_data(required_fields,object_):
    
    sf = Salesforce(username = username_bsna,
                    password = password_bsna,
                    security_token = token,
                    sandbox = isSandbox,
                    proxies = proxies)
    
    # get api fields from e3 instance, validate and turn into query
    
    descString = f'sf.{object_}.describe()'
    desc = eval(descString)
    bsna_fields = [(x.get('name'), x.get('picklistValues')) for x in desc['fields']]
    bsna_df = pd.DataFrame(bsna_fields)
    # call SIT instance to find api keys of all related objects

    missingCols = set(required_fields['FieldName(API)BSNA'].tolist()) - set(bsna_df[0].tolist())
    # columns that are missing in the new instance

    bsnaScope = ','.join(list(set(required_fields['FieldName(API)BSNA'].tolist()).intersection(set(bsna_df[0].tolist()))))
    
    global bsnaScopeList
    bsnaScopeList = list(set(required_fields['FieldName(API)BSNA'].tolist()).intersection(set(bsna_df[0].tolist())))
    #query known existing values from object that are also in the Mapping document, all others will be reported as missing.

    bsna = sf.query_all("SELECT DM_ID__c," + bsnaScope + ' FROM ' + object_ + ' WHERE DM_Load_Date__c > 2019-06-12T00:00:00Z')
    bsna = pd.DataFrame.from_records(bsna['records'])
    
    return {'bsna':bsna}, missingCols

In [112]:
# read in mapping document for contact IF some values are picklist in STM
# picklist document relectes any changes to field values

def map_picklist(e3, object_):
    
    # a function for trying to map picklist values from document
    def tryDict(row):
        if row is None:
            try:
                return d[x][np.nan]
            except:
                return row
        else:      
            try:
                return d[x][row]
            except:
                return row
    

    try:
        map_doc = pd.read_excel(r'C:\Users\jschlajo\Desktop\Migration_Picklists\E3ObjectsPicklistMappingMay.xlsx', sheet_name = object_)
        pd.options.display.max_rows = 200
        map_doc.replace('currencyisocode', 'CurrencyIsoCode', inplace=True)

        d = defaultdict(dict)

        for x in map_doc.values:
            d[x[5]][x[6]] = x[11]
            #create mapping dictionary from picklist Document


        keys = d.keys()

        for x in d.keys():
            if pd.isnull(x):
                break
            else:
                e3[x] = e3[x].map(tryDict)
                # map new values to reaplce old values. Next step, make sure all currnet values = mapped new values
    except:
        # object is not in picklist document
        e3 = e3
                
    return e3

In [114]:
def e3_data(required_fields,object_):
    
    # same logic as bsna query
       
    sf = Salesforce(username = username_e3,
                    password = password_e3,
                    security_token = token,
                    sandbox = isSandbox,
                    proxies = proxies)
    

    descString = f'sf.{object_}.describe()'
    desc = eval(descString)
    e3_fields = [(x.get('name'), x.get('picklistValues')) for x in desc['fields']]
    e3_df = pd.DataFrame(e3_fields)

    missingCols = set(required_fields['FieldName(API)E3'].tolist()) - set(e3_df[0].tolist())

    e3Scope = ','.join(list(set(required_fields['FieldName(API)E3'].tolist()).intersection(set(e3_df[0].tolist()))))
    global e3ScopeList
    e3ScopeList = list(set(required_fields['FieldName(API)E3'].tolist()).intersection(set(e3_df[0].tolist())))

    e3 = sf.query_all("SELECT " + e3Scope + ' FROM ' + object_)
    e3 = pd.DataFrame.from_records(e3['records'])
    
    # excute function for mapping picklist changes to the data
    e3 = map_picklist(e3, object_)
    
    return {'e3':e3}, missingCols, e3ScopeList

In [102]:
def concurrent_query():
    # a function using concurrent futures to run both instance queries simulateneously
    
    functions = ['e3_data(required_fields,object_)','bsna_data(required_fields,object_)']
    
    e3Data = []
    e3ColErr = []
    
    bsnaData = []
    bsnaColErr = []

    def sf_query(index):
            result = eval(functions[index])
            if 'e3' in functions[index]:
                e3Data.append(result[0])
                e3ColErr.append(result[0])
            else:  
                bsnaData.append(result[0])
                bsnaColErr.append(result[1])

    def main_2():
        with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
            results = [executor.submit(sf_query,x) for x in range(0,2)]
    
    run = main_2()
            
    return e3Data[0], e3ColErr, bsnaData[0], bsnaColErr

In [151]:
def compare_data():
    
    data = concurrent_query()
    
    e3 = data[0]['e3']
    bsna = data[2]['bsna']
    
    
    # scope of columns to check taken from STM Mapping document
    scopeDf = required_fields[(required_fields['FieldName(API)E3'].isin(e3ScopeList)) & (required_fields['FieldName(API)BSNA'].isin(bsnaScopeList))]
    scopeDf_mapping = dict(zip(scopeDf['FieldName(API)E3'],scopeDf['FieldName(API)BSNA'] ))

    # if columns have same name in new instance, append _x , _y to differentiate. Other wise just compare the new col with old
    compare_tuples = []
    for k,v in scopeDf_mapping.items():
        if k==v:
            check = [k+'_x',v+'_y']
            compare_tuples.append(check)
        else:
            check = [k,v]
            compare_tuples.append(check)

    #create mapping to compare like columns once dataframes are merged 

    compare = e3.merge(bsna, left_on='Id', right_on='DM_ID__c', how='inner')
    
    # list allows me to look at columns side by side for comparison, many times small changes are acceptable and need to be manually glossed over
    orgList = list(itertools.chain.from_iterable(compare_tuples))
    pd.options.display.max_columns = 100
    compare = compare.reindex_axis(orgList, axis=1)
    compare.fillna('', inplace=True)
    
    #other times only perfect data migration is acceptable. this will reuturn a dataframe which value counts of mis-matches for every column can be looked at

    # a function to check for perfectly migrated data
    def data_check(compare):

        frames = []
        for x in compare_tuples:
            df = compare[[x[0], x[1], 'Id_x', 'Id_y']]
            if x[0] != 'Id_x':
                df = df[df[x[0]] != df[x[1]]]
                df.columns = ['E3', 'BSNA', 'E3_id', 'BSNA_id']
                df['object'] = str(x)
                frames.append(df)
            else:
                pass

        return pd.concat(frames)
    
    perfect_compare = data_check(compare)
    
    
    return compare, perfect_compare

In [152]:
test = compare_data()

In [150]:
test[0].head()[['ActualCost_x', 'ActualCost_y', 'AmountAllOpportunities_x',
       'AmountAllOpportunities_y', 'AmountWonOpportunities_x',
       'AmountWonOpportunities_y', 'BudgetedCost_x', 'BudgetedCost_y']]

,ActualCost_x,ActualCost_y,AmountAllOpportunities_x,AmountAllOpportunities_y,AmountWonOpportunities_x,AmountWonOpportunities_y,BudgetedCost_x,BudgetedCost_y
0,,,228419.848379,0.0,51219.018379,0.0,0,0
1,,,20000.000000,0.0,0.000000,0.0,0,0
2,,,0.000000,0.0,0.000000,0.0,0,0
3,0,0,124835.799500,0.0,56237.000000,0.0,200,200
4,40000,40000,0.000000,0.0,0.000000,0.0,40000,40000


In [147]:
test[1]['object'].value_counts().head(10)

['OwnerId_x', 'OwnerId_y']                                                    385
['HierarchyNumberOfLeads_x', 'HierarchyNumberOfLeads_y']                      350
['NumberOfLeads_x', 'NumberOfLeads_y']                                        347
['HierarchyNumberOfResponses_x', 'HierarchyNumberOfResponses_y']              262
['NumberOfResponses_x', 'NumberOfResponses_y']                                260
['HierarchyNumberOfOpportunities_x', 'HierarchyNumberOfOpportunities_y']      249
['HierarchyNumberOfConvertedLeads_x', 'HierarchyNumberOfConvertedLeads_y']    246
['NumberOfOpportunities_x', 'NumberOfOpportunities_y']                        246
['NumberOfConvertedLeads_x', 'NumberOfConvertedLeads_y']                      244
['HierarchyAmountAllOpportunities_x', 'HierarchyAmountAllOpportunities_y']    236
Name: object, dtype: int64